Declaration

In [46]:
import numpy as np
import os
import urllib.request
from sklearn import mixture
from scipy.spatial import distance
import math
import statistics 

CUT_OFF_IN_PERCENT = 0.02

# Clustering data set
# http://cs.joensuu.fi/sipu/datasets/
def download_data_set(file_url = 'http://cs.joensuu.fi/sipu/datasets/s1.txt'):
  local_filename, headers = urllib.request.urlretrieve(url = file_url)
  return np.delete(np.loadtxt(local_filename), 2, 1)

def gmm_selection(data_set):
  lowest_bic = np.infty
  bic = []
  n_components_range = range(1, 10)
  cv_types = ['spherical', 'tied', 'diag', 'full']
  for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a Gaussian mixture with EM
        gmm = mixture.GaussianMixture(n_components=n_components, covariance_type=cv_type)
        gmm.fit(data_set)
        bic.append(gmm.bic(data_set))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm
  return best_gmm

# Get all euclid distance between points with keeping their respective order
def get_all_distances(data_set):
  arr_leng, dims = data_set.shape
  all_distances = []
  for i in range(arr_leng):
    current_point_distances = []
    for j in range(arr_leng):
      if i == j:
        continue
      current_point_distances.append(distance.euclidean(data_set[i], data_set[j]))
    all_distances.append(current_point_distances)
  return np.array(all_distances)

def get_cut_off_distance(distances_data):
  arr_leng, dims = distances_data.shape
  # Get number of neighbors
  number_of_items = math.ceil(CUT_OFF_IN_PERCENT * dims)
  all_distances = []
  for i in range(arr_leng):
    point_distance = np.array(distances_data[i])
    # Get number_of_items smallest distance
    point_distance = np.partition(point_distance,number_of_items)[:number_of_items]
    for dist in point_distance:
      all_distances.append(dist)
  return statistics.mean(all_distances) 

def get_density_of_point(point_distance, cut_off_distance):
  arr_leng = len(point_distance)
  result = 0
  for i in range(arr_leng):
    if point_distance[i] < cut_off_distance:
      result += 1
  return result

def get_densities(distances_data, cut_off_distance):
  arr_leng, dims = distances_data.shape
  result = []
  for i in range(arr_leng):
    result.append(get_density_of_point(distances_data[i], cut_off_distance))
  return np.array(result)

Download S1 data set, get all distances arrays and calculate cut off distance

In [ ]:
r15 = download_data_set("http://cs.joensuu.fi/sipu/datasets/R15.txt")
print(r15)
all_distances = get_all_distances(r15)
cut_off_distance = get_cut_off_distance(all_distances)
print(all_distances.shape)

In [ ]:
abc = np.array([1,2,3,4,5,6,7,8,9,10,11,12])
vvv = gmm_selection(np.expand_dims(abc,1))
print(vvv)
print(vvv.predict(np.expand_dims(abc,1)))



In [ ]:
print(all_distances)
print(cut_off_distance)
densities = get_densities(all_distances, cut_off_distance * 0.5)
print(densities)